# Inicialização e pré-processamento de dados

## Manipulação de dados

#### Guias:
* [package dplyr](https://dplyr.tidyverse.org/)
* [package apply](https://stackoverflow.com/questions/3505701/grouping-functions-tapply-by-aggregate-and-the-apply-family?rq=1)
* [data.table vs dplyr](https://stackoverflow.com/questions/21435339/data-table-vs-dplyr-can-one-do-something-well-the-other-cant-or-does-poorly?rq=1)

#### Sampling:

Procedeu-se a um sampling de em que reduziu o dataset para 25% do seu tamanho original:
    
1. Calcular a percentagem de ocorrência de cada primary type no dataset completo.
2. Proceder-se à sample de 25% do dataset.
3. Calcular a percentagem de ocorrência de cada primary type no dataset partido.
4. Comparar as percentagens de modo a perceber se a distribuição alterou muito.
    
[Porquê o sampling de 25%](https://www.researchgate.net/post/How_to_sample_a_smaller_data_set_from_a_very_large_data_set)?

Para que o sampling fosse eficaz na redução do tempo de computação dos algoritmos são feitos **dois** samplings distintos:
* Uma primeira sample que contém os dados necessários para a resposta a perguntas sobre os diferentes tipos de crimes, as localizações gerais, as suas relações, relevâncias e evolução temporal.
* Uma segunda sample que permita responder a perguntas relacionadas com a localização geográfico e evolução temporal da mesma.

## Bibliotecas necessárias:

In [2]:
require(data.table) #fread / apply
library(tidyverse) #readr / dplyr / ggplot2

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.1.1       v purrr   0.3.2  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v ggplot2 3.1.1       v forcats 0.4.0  
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::between()   masks data.table::between()
x dplyr::filter()    masks stats::filter()
x dplyr::first()     masks data.table::first()
x dplyr::lag()       masks stats::lag()
x dplyr::last()      masks data.table::last()
x purrr::transpose() masks data.table::transpose()


## Leitura de dados:
Leitura dos diferentes ficheiros utilizando *read_csv* com indicação de tipos de colunas (para melhoria de tempo de leitura)

In [3]:
types = "iic?cccccllinnncnni?nn?"
  
crimes_2001_2004 = read_csv(file = "datasets/Chicago_Crimes_2001_to_2004.csv", col_types = types)
crimes_2005_2007 = read_csv(file = "datasets/Chicago_Crimes_2005_to_2007.csv", col_types = types)
crimes_2008_2011 = read_csv(file = "datasets/Chicago_Crimes_2008_to_2011.csv", col_types = types)
crimes_2012_2017 = read_csv(file = "datasets/Chicago_Crimes_2012_to_2017.csv", col_types = types)

Warning message:
"Missing column names filled in: 'X1' [1]"Warning message:
"14 parsing failures.
    row      col           expected               actual                                       file
1513590 Arrest   1/0/T/F/TRUE/FALSE Location Description 'datasets/Chicago_Crimes_2001_to_2004.csv'
1513590 Domestic 1/0/T/F/TRUE/FALSE Arrest               'datasets/Chicago_Crimes_2001_to_2004.csv'
1513590 Beat     an integer         Domestic             'datasets/Chicago_Crimes_2001_to_2004.csv'
1513590 District a number           Beat                 'datasets/Chicago_Crimes_2001_to_2004.csv'
1513590 Ward     a number           District             'datasets/Chicago_Crimes_2001_to_2004.csv'
....... ........ .................. .................... ..........................................
See problems(...) for more details.
"Warning message:
"Missing column names filled in: 'X1' [1]"Warning message:
"1 parsing failure.
   row      col           expected actual                            

Eliminação de coluna inútil:

In [4]:
crimes_2001_2004 = crimes_2001_2004[, !(names(crimes_2001_2004) %in% "X1")]
crimes_2005_2007 = crimes_2005_2007[, !(names(crimes_2005_2007) %in% "X1")]
crimes_2008_2011 = crimes_2008_2011[, !(names(crimes_2008_2011) %in% "X1")]
crimes_2012_2017 = crimes_2012_2017[, !(names(crimes_2012_2017) %in% "X1")]
head(crimes_2001_2004)

ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
4786321,HM399414,01/01/2004 12:01:00 AM,082XX S COLES AVE,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,FALSE,FALSE,...,7,46,06,NA,NA,2004,08/17/2015 03:03:40 PM,NA,NA,NA
4676906,HM278933,03/01/2003 12:00:00 AM,004XX W 42ND PL,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,FALSE,TRUE,...,11,61,26,1173974,1876757,2003,04/15/2016 08:55:02 AM,41.81723,-87.63733,"(41.817229156, -87.637328162)"
4789749,HM402220,06/20/2004 11:00:00 AM,025XX N KIMBALL AVE,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,FALSE,FALSE,...,35,22,20,NA,NA,2004,08/17/2015 03:03:40 PM,NA,NA,NA
4789765,HM402058,12/30/2004 08:00:00 PM,045XX W MONTANA ST,0840,THEFT,FINANCIAL ID THEFT: OVER $300,OTHER,FALSE,FALSE,...,31,20,06,NA,NA,2004,08/17/2015 03:03:40 PM,NA,NA,NA
4677901,HM275615,05/01/2003 01:00:00 AM,111XX S NORMAL AVE,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,FALSE,FALSE,...,34,49,06,1174948,1831051,2003,04/15/2016 08:55:02 AM,41.69178,-87.63512,"(41.691784636, -87.635115968)"
4838048,HM440266,08/01/2004 12:01:00 AM,012XX S HARDING AVE,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,APARTMENT,FALSE,FALSE,...,24,29,06,NA,NA,2004,08/17/2015 03:03:40 PM,NA,NA,NA


## Sampling do dataset

* Eliminação colunas que não serão utilizadas para responder a perguntas sobre os crimes, sem ter em conta a localização.
* Tabelas de frequência das diferentes ativiades criminais existentes.
* Redução do dataset para 25% do seu tamanho.
* Teste à igualdade entre as duas distribuições. (dar valores numéricos aos Primary Type e proceder a testes de hipótese à mediana)

Esta sample será a **primeira** das duas samples que serão feitas.

O dataset irá ficar com as colunas **Date**, **Primary_Type**, **Description**, **Location_Description**, **Arrest**, **Domestic**, **District** e **Year**.

In [18]:
#Eliminar colunas não necessárias
non_sampled_2001_2004 = crimes_2001_2004[-c(1:2, 4:5, 11, 13:17, 19:22)]

colnames(non_sampled_2001_2004)[2] = "Primary_Type"; colnames(non_sampled_2001_2004)[4] = "Location_Description"

#Tipos de crime com percentagem 0:
#DOMESTIC VIOLENCE / IUCR (erro no df) / NON-CRIMINAL / OBSCENITY / OTHER NARCOTIC VIOLATION / PUBLIC INDECENCY / RITUALISM
non_sampled_2001_2004 = filter(non_sampled_2001_2004, !Primary_Type %in% c("DOMESTIC VIOLENCE", "IUCR", "NON-CRIMINAL", "OBSCENITY", "OTHER NARCOTIC VIOLATION", "PUBLIC INDECENCY", "RITUALISM"))

#Tabela de frequência do Primary_Type e Description
sample_2001_2004_table = lapply(non_sampled_2001_2004["Primary_Type"], table)

crime_types = as.data.frame(sample_2001_2004_table$Primary_Type)
colnames(crime_types)[1] = "Type"

#options("scipen"=100, "digits"=4) #não mostrar potencias de 10
#percentagem de ocorrencia de cada crime
crime_types = crime_types %>% 
  mutate(Type, percent = round((Freq / nrow(non_sampled_2001_2004)) * 100, 2))

crime_types

Type,Freq,percent
ARSON,3828,0.20
ASSAULT,121417,6.31
BATTERY,363538,18.90
BURGLARY,101902,5.30
CRIM SEXUAL ASSAULT,7079,0.37
CRIMINAL DAMAGE,220298,11.45
CRIMINAL TRESPASS,57770,3.00
DECEPTIVE PRACTICE,55751,2.90
GAMBLING,4051,0.21
HOMICIDE,3172,0.16


In [19]:
#sampling de 25%
nrow(non_sampled_2001_2004)
sample_2001_2004 = non_sampled_2001_2004[seq(1, nrow(non_sampled_2001_2004), 4), ]
nrow(sample_2001_2004)


#Verificação de percentagens
sample_2001_2004_table = lapply(sample_2001_2004["Primary_Type"], table)

sampled_crime_types = as.data.frame(sample_2001_2004_table)
colnames(sampled_crime_types) = c("Type", "Freq")

sampled_crime_types = sampled_crime_types %>% 
  mutate(Type, percent = round((Freq / nrow(sample_2001_2004)) * 100, 2))

sampled_crime_types

[1] 1923372

[1] 480843

Type,Freq,percent
ARSON,967,0.20
ASSAULT,30315,6.30
BATTERY,90734,18.87
BURGLARY,25436,5.29
CRIM SEXUAL ASSAULT,1701,0.35
CRIMINAL DAMAGE,55089,11.46
CRIMINAL TRESPASS,14342,2.98
DECEPTIVE PRACTICE,13838,2.88
GAMBLING,1007,0.21
HOMICIDE,788,0.16


In [21]:
#Transformar valores categóricos de Primary_Type em valores numericos
numeric_primary_type_pop1 = unclass(as.factor(non_sampled_2001_2004$Primary_Type))
numeric_primary_type_pop2 = unclass(as.factor(sample_2001_2004$Primary_Type))

mean(numeric_primary_type_pop1)
mean(numeric_primary_type_pop2)

median(numeric_primary_type_pop1)
median(numeric_primary_type_pop2)

sd(numeric_primary_type_pop1)
sd(numeric_primary_type_pop2)

[1] 12.34638

[1] 12.36612

[1] 15

[1] 15

[1] 8.423311

[1] 8.429137

Comparando as percentagens da amostra e da população é possível perceber que estas são bastante parecidas.

Para além disso são verificados outros fatores:
* A diferença absoluta das médias dos valores dos crimes é de 0.02
* As médias da amostra e da população são iguais (15)
* Os desvios padrão dos valores tem uma diferença absoluta de 0.006

Verificando que as diferenças entre estes valores são muito baixas ou inexistentes, não há evidência estatistica que contrarie o facto desta amostra ser representativa desta população.

O processo de pré-processamento **repete-se** para os restastes três datasets.

1. Repetir o processo feito anteriormente
2. Juntar as quatro tabelas de cada dataset numa só
3. Fazer a tabela de frequencias absolutas e relativas
4. Fazer um gráfico que demonstre a percentagem de cada tipo de crime